# Power Capacity Expansion Problem

Install packages, setup Gurobi license and read in data. $\color{red}{\text{Add the codes for your license.}}$

In [ ]:
!pip install gurobipy

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
from google.colab import drive

# allow access to google drive
drive.mount('/content/drive')

generators = pd.read_csv("drive/MyDrive/Colab Notebooks/CE4110_6250/expansion_data/generators_for_expansion.csv")
demand = pd.read_csv("drive/MyDrive/Colab Notebooks/CE4110_6250/expansion_data/demand_for_expansion.csv")
capacity_factors = pd.read_csv("drive/MyDrive/Colab Notebooks/CE4110_6250/expansion_data/wind_solar_for_expansion.csv")

# create environment with Gurobi academic license
params = {
"WLSACCESSID": ,
"WLSSECRET": ,
"LICENSEID": ,
}
env = gp.Env(params=params)

## Optimization only considering thermal power plants (from class)

In [ ]:
# number of generators and hours
G = len(generators["G"].iloc[0:-2]) # ignore solar and wind for now
H = len(demand["Hour"])

# parameters
NSEcost = 9000
demands = demand["Demand"].values
FixedCosts = generators['FixedCost'][0:-2].values
VariableCosts = generators["VarCost"][0:-2].values

# Create the model within the Gurobi environment
model = gp.Model('Capacity Expansion',env=env)

# decision variables
CAP = model.addVars(G, vtype=GRB.CONTINUOUS, name="CAP")
GEN = model.addVars(G, H, vtype=GRB.CONTINUOUS, name="GEN")
NSE = model.addVars(H, vtype=GRB.CONTINUOUS, name="NSE")

#constraints
demand_constraint = model.addConstrs((gp.quicksum(GEN[g,h] for g in range(G)) + NSE[h] == demands[h])
                                     for h in range(H))
capacity_constraint = model.addConstrs((GEN[g,h] <= CAP[g])
                                       for g in range(G) for h in range(H))

#objective function
fixed_cost = gp.quicksum(FixedCosts[g] * CAP[g] for g in range(G))
variable_cost = gp.quicksum(VariableCosts[g] * GEN[g,h] for g in range(G) for h in range(H))
curtailment_cost = gp.quicksum(NSEcost * NSE[h] for h in range(H))

model.setObjective(fixed_cost + variable_cost + curtailment_cost)

model.write('capacity_expansion.lp')
model.optimize()

In [ ]:
# get results
rows = ["Hour" + str(h) for h in range(H)]
gen = pd.DataFrame(columns=generators["G"].iloc[0:-2], index=rows, data=0.0)
for h in range(H):
    for g in range(G):
        gen.loc["Hour"+str(h), generators["G"].iloc[g]] = GEN[g,h].x

rows = generators["G"].iloc[0:G]
cap = pd.DataFrame(columns=["Capacity"], index=rows, data=0.0)
for g in range(G):
  cap["Capacity"].iloc[g] = CAP[g].x

rows = ["Hour" + str(h) for h in range(H)]
nse = pd.DataFrame(columns=["NSE"], index=rows, data=0.0)
for h in range(H):
  nse["NSE"].iloc[h] = NSE[h].x

# summarize the results in a table with the total capacity of each resource in MW,
# the percent of capacity from each resource
# the total generation from each resource in Gwh,
# and the percent of total generation from each resource
generation = np.sum(gen,0) # total generation from each resource
MWh_share = generation/np.sum(demand["Demand"])*100 # percent of generation from each resource
cap_share = cap/np.max(demand["Demand"])*100 # percent of capacity from each resource
results = pd.DataFrame({"Resource": generators["G"].iloc[0:-2].values, "MW": cap["Capacity"].values,
                        "Percent_MW": cap_share["Capacity"].values, "GWh": generation.values/1000,
                        "Percent_GWh": MWh_share.values})

# Calculate how much non-served energy there was and add to results
# The maximum MW of non-served energy is the difference
# between peak demand and total installed generation capacity
NSE_MW = np.max(nse,0)
# The total MWh of non-served energy is the difference between
# total demand and total generation
NSE_MWh = np.sum(nse,0)
# Add a new row of data to the end of the results DataFrame
results.loc[len(results.index)] = ["NSE", NSE_MW["NSE"], NSE_MW["NSE"]/np.max(demand["Demand"])*100,
                                   NSE_MWh["NSE"]/1000, NSE_MWh["NSE"]/np.sum(demand["Demand"])*100]
results

## 2. Optimization with renewables

Create a new model, model2 that includes renewables.
$\color{red}{\text{The code below is complete.}}$

In [ ]:
# parameters
capacity_factors = capacity_factors.iloc[:,1::] # drop hours column
G = len(generators["G"])
RE = 2 # number of renewables
FixedCosts = generators['FixedCost'].values
VariableCosts = generators["VarCost"].values

# Create the model within the Gurobi environment
model2 = gp.Model('Capacity Expansion with Renewables',env=env)

$\color{red}{\text{Edit the code below to adapt the decision variables, constraints, and objective function with renewables included.}}$

In [ ]:
# decision variables


# constraints


# objective function

model2.setObjective(fixed_cost + variable_cost + curtailment_cost)
model2.write('capacity_expansion_renewables.lp')
model2.optimize()

$\color{red}{\text{Print the results. The code below is complete.}}$

In [ ]:
# get results
rows = ["Hour" + str(h) for h in range(H)]
gen = pd.DataFrame(columns=generators["G"], index=rows, data=0.0)
for h in range(H):
    for g in range(G):
        gen.loc["Hour"+str(h), generators["G"].iloc[g]] = GEN[g,h].x

rows = generators["G"].iloc[0:G]
cap = pd.DataFrame(columns=["Capacity"], index=rows, data=0.0)
for g in range(G):
  cap["Capacity"].iloc[g] = CAP[g].x

rows = ["Hour" + str(h) for h in range(H)]
nse = pd.DataFrame(columns=["NSE"], index=rows, data=0.0)
for h in range(H):
  nse["NSE"].iloc[h] = NSE[h].x

# summarize the results in a table with the total capacity of each resource in MW,
# the percent of capacity from each resource
# the total generation from each resource in Gwh,
# and the percent of total generation from each resource
generation = np.sum(gen,0) # total generation from each resource
MWh_share = generation/np.sum(demand["Demand"])*100 # percent of generation from each resource
cap_share = cap/np.max(demand["Demand"])*100 # percent of capacity from each resource
results2 = pd.DataFrame({"Resource": generators["G"].values, "MW": cap["Capacity"].values,
                        "Percent_MW": cap_share["Capacity"].values, "GWh": generation.values/1000,
                        "Percent_GWh": MWh_share.values})

# Calculate how much non-served energy there was and add to results
# The maximum MW of non-served energy is the difference
# between peak demand and total installed generation capacity
NSE_MW = np.max(nse,0)
# The total MWh of non-served energy is the difference between
# total demand and total generation
NSE_MWh = np.sum(nse,0)
# Add a new row of data to the end of the results DataFrame
results2.loc[len(results2.index)] = ["NSE", NSE_MW["NSE"], NSE_MW["NSE"]/np.max(demand["Demand"])*100,
                                   NSE_MWh["NSE"]/1000, NSE_MWh["NSE"]/np.sum(demand["Demand"])*100]
results2

## 3. Optimization considering renewables and retirements

Add information on the old generators to the data frame.
$\color{red}{\text{The code below is complete.}}$

In [ ]:
# Add data on old generators to dataframe
generators.loc[len(generators.index)] = ["Old_CCGT","Existing CCGT",0,40000,5,7.5,4,0,0,0,0,40000,30]
generators.loc[len(generators.index)] = ["Old_CT","Existing CT",0,30000,11,11.0,4,0,0,0,0,30000,55]

# Set installed capacity for existing CCGTs:
ExistingCap_CCGT = 1260 # Approximate actual existing capacity in SDGE
ExistingCap_CT = 925 # Approximate actual existing capacity in SDGE
# Add new column to generators Data Frame
generators["ExistingCap"] = [0,0,0,0,0,0,ExistingCap_CCGT,ExistingCap_CT];
generators

Create a new model, model3, that includes old generators. $\color{red}{\text{The code below is complete.}}$

In [ ]:
# parameters
FixedCosts = generators['FixedCost'].values
VariableCosts = generators["VarCost"].values
ExistingCap = generators["ExistingCap"].values
G = len(generators["G"])
GOLD = 2
GNEW = G - 2

# Create the model within the Gurobi environment
model3 = gp.Model('Capacity Expansion and Retirement',env=env)

$\color{red}{\text{Edit the code below to adapt the decision variables, constraints, and objective function with renewables and old generators included.}}$

In [ ]:
# decision variables


# constraints


# objective function


model3.setObjective(new_fixed_cost + old_fixed_cost + variable_cost + curtailment_cost)
model3.write('capacity_expansion_retirement.lp')
model3.optimize()

$\color{red}{\text{Print the results. The code below is complete.}}$

In [ ]:
# get results
rows = ["Hour" + str(h) for h in range(H)]
gen = pd.DataFrame(columns=generators["G"], index=rows, data=0.0)
for h in range(H):
    for g in range(G):
        gen.loc["Hour"+str(h), generators["G"].iloc[g]] = GEN[g,h].x

rows = generators["G"].iloc[0:GNEW]
cap = pd.DataFrame(columns=["Capacity"], index=rows, data=0.0)
for g in range(GNEW):
  cap["Capacity"].iloc[g] = CAP[g].x

rows = generators["G"].iloc[GNEW::]
ret = pd.DataFrame(columns=["Retired"], index=rows, data=0.0)
for g in range(GOLD):
  ret["Retired"].iloc[g] = RET[g].x

rows = ["Hour" + str(h) for h in range(H)]
nse = pd.DataFrame(columns=["NSE"], index=rows, data=0.0)
for h in range(H):
  nse["NSE"].iloc[h] = NSE[h].x

# summarize the results in a table with the total capacity of each resource in MW,
# the percent of capacity from each resource
# the total generation from each resource in Gwh,
# and the percent of total generation from each resource
generation = np.sum(gen,0) # total generation from each resource
MWh_share = generation/np.sum(demand["Demand"])*100 # percent of generation from each resource
cap_share = cap/np.max(demand["Demand"])*100 # percent of capacity from each resource
results3 = pd.DataFrame({"Resource": generators["G"].iloc[0:GNEW].values, "MW": cap["Capacity"].values,
                        "Percent_MW": cap_share["Capacity"].values, "GWh": generation[0:GNEW].values/1000,
                        "Percent_GWh": MWh_share[0:GNEW].values, "Retirement": np.zeros([GNEW])})
for i in range(2):
  results3.loc[len(results3.index)] = [generators["G"].iloc[-2+i], ExistingCap[GNEW+i] - ret.iloc[i].values[0],
                                   (ExistingCap[GNEW+i] - ret.iloc[i].values[0])/np.max(demand["Demand"])*100,
                                   generation[GNEW+i]/1000, generation[GNEW+i]/np.sum(demand["Demand"])*100,
                                   ret.iloc[i].values[0]]

# Calculate how much non-served energy there was and add to results
# The maximum MW of non-served energy is the difference
# between peak demand and total installed generation capacity
NSE_MW = np.max(nse,0)
# The total MWh of non-served energy is the difference between
# total demand and total generation
NSE_MWh = np.sum(nse,0)
# Add a new row of data to the end of the results DataFrame
results3.loc[len(results3.index)] = ["NSE", NSE_MW["NSE"], NSE_MW["NSE"]/np.max(demand["Demand"])*100,
                                   NSE_MWh["NSE"]/1000, NSE_MWh["NSE"]/np.sum(demand["Demand"])*100, 0]
results3